# [sec0] load raw_images


In [ ]:
import numpy as np

In [ ]:
frames=np.load("../../../whimh2.0/outfile_x.npy")
y_train=np.load("../../../whimh2.0/outfile_y.npy").astype(int)

In [ ]:
print "MH : " +str((y_train==0).sum() )
print "NM : " +str((y_train==1).sum())
print "NH : " +str((y_train==2).sum())
print "Me : " +str((y_train==3).sum())
print "So : " +str((y_train==4).sum())

In [ ]:
# NM labels too much
arrMH=np.where(y_train==0)[0]
arrNM=np.where(y_train==1)[0]
arrNH=np.where(y_train==2)[0]
arrME=np.where(y_train==3)[0]
arrSO=np.where(y_train==4)[0]
max_=len(arrNM)
indx=np.random.randint(max_, size=1000)
newarr=np.concatenate((arrMH,arrNM[indx],arrNH,arrME,arrSO))

In [ ]:
frames=frames[newarr]
y_train=y_train[newarr]
print frames.shape
print y_train.shape

# [sec0-1] create rotate images

In [ ]:
import cv2
import random
def getrotation_img(image,angel):
    rows,cols,c = image.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angel,1)
    dst = cv2.warpAffine(image,M,(cols,rows))
    return dst

def rotation_all(X_train,angle):
    for i in xrange(X_train.shape[0]):
        angle+=random.randint(-22 ,22)
        if i==0:
            X_train_temp=np.reshape(getrotation_img(X_train[i],angle),[-1,32,32,3])
        else:
            temp=np.reshape(getrotation_img(X_train[i],angle),[-1,32,32,3])
            X_train_temp=np.concatenate((X_train_temp, temp), axis=0)
    print "the rotation output : "+str(X_train_temp.shape)
    return X_train_temp

In [ ]:
# training set
X_train=frames
X_train_0=rotation_all(frames,0)
X_train_0_1=rotation_all(frames,0)
X_train_45=rotation_all(frames,45)
X_train_45_1=rotation_all(frames,45)
X_train_90=rotation_all(frames,90)
X_train_90_1=rotation_all(frames,90)
X_train_135=rotation_all(frames,135)
X_train_135_1=rotation_all(frames,135)
X_train_180=rotation_all(frames,180)
X_train_180_1=rotation_all(frames,180)
X_train_180_2=rotation_all(frames,180)
X_train_225=rotation_all(frames,225)
X_train_225_1=rotation_all(frames,225)
X_train_270_1=rotation_all(frames,270)
X_train_270=rotation_all(frames,270)
X_train_315_1=rotation_all(frames,315)
X_train_315=rotation_all(frames,315)

In [ ]:
frames=np.concatenate((X_train,X_train_0,X_train_0_1,X_train_45,X_train_45_1,X_train_90,X_train_90_1,\
                       X_train_135,X_train_180,X_train_225,X_train_135_1,X_train_180_1,X_train_180_2
                       ,X_train_225_1,X_train_270,X_train_315,X_train_270_1,X_train_315_1),axis=0)
print frames.shape

y_train=np.concatenate((y_train,y_train,y_train,y_train,y_train,y_train,\
                        y_train,y_train,y_train,y_train,y_train,y_train,\
                       y_train,y_train,y_train,y_train,y_train,y_train),axis=0)
print y_train.shape

# [sec0-2] save it

In [ ]:
np.save("rotated_x",frames)
np.save("rotated_y",y_train)

# [Sec1-0] train raw images with naive cnn first ~85%

In [1]:
from layer_wrapper_stn import naive_cnn,evaluation,split_data
import numpy as np
import tensorflow as tf

In [2]:
# split dataset
frames = np.load("../../../whimh2.0/outfile_x.npy")
y_train = np.load("../../../whimh2.0/outfile_y.npy").astype(int)
# data is a dictionary with key X_train,y_train ,X_val,y_val
# NM labels too much
arrMH=np.where(y_train==0)[0]
arrNM=np.where(y_train==1)[0]
arrNH=np.where(y_train==2)[0]
arrME=np.where(y_train==3)[0]
arrSO=np.where(y_train==4)[0]
max_=len(arrNM)
indx=np.random.randint(max_, size=1000)
newarr=np.concatenate((arrMH,arrNM[indx],arrNH,arrME,arrSO))
frames=frames[newarr]
y_train=y_train[newarr]
print frames.shape
print y_train.shape


(2231, 32, 32, 3)
(2231,)


In [3]:
data = split_data(y_train,frames)

there are 1784 images in training set
there are 447 images in testing set


In [4]:
# concate a naive cnn
[x_tensor,y_tf,y_tf_2,y_logits,regu,keep_prob] = naive_cnn()
[train_step,merged,accuracy,cross_entropy,regular_lambda,learning_rate] = evaluation(y_logits,y_tf_2,keep_prob,regu,decay_step=500)

In [10]:
# create session and initialize all variables, and create writer
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#train_writer = tf.summary.FileWriter('/tmp/naive_cnn_face/train',sess.graph)
#test_writer = tf.summary.FileWriter('/tmp/naive_cnn_face/test')
count=0

In [12]:
# %% We'll now train in minibatches and report accuracy, loss:
iter_per_epoch = 100
n_epochs = 1000
train_size = len(data['X_train'])   
regular_lambda_=1e-1
indices = np.linspace(0, train_size-1, iter_per_epoch)
indices = indices.astype('int')

for epoch_i in range(n_epochs):
    for iter_i in range(iter_per_epoch - 1):
        batch_xs = data['X_train'][indices[iter_i]:indices[iter_i+1]]
        batch_ys = data['y_train'][indices[iter_i]:indices[iter_i+1]]
        sess.run(train_step, feed_dict={
            x_tensor: batch_xs, y_tf: batch_ys, learning_rate:1e-4, keep_prob:0.3, regular_lambda:regular_lambda_})
    
        if iter_i % 99 ==0and epoch_i%20 == 0:
            summary,loss,accuracy_ = sess.run([merged,cross_entropy,accuracy],
                            feed_dict={
                                x_tensor: data['X_train'],
                                y_tf: data['y_train'],
                                regular_lambda: regular_lambda_,
                                keep_prob:1
                            })

            #train_writer.add_summary(summary, count)
 
            summary,loss_val,accuracy_val = sess.run([merged,cross_entropy,accuracy],
                            feed_dict={
                                x_tensor: data['X_val'],
                                y_tf: data['y_val'],
                                regular_lambda: regular_lambda_,
                                keep_prob:1
                            })
            #test_writer.add_summary(summary, count)

            print('training acc: ' + str(accuracy_) +' testing acc :' +str(accuracy_val)+ 
                  ' /training Loss: ' + str(loss) +' testing loss :' +str(loss_val))
            count+=1
        

training acc: 0.994395 testing acc :0.8434 /training Loss: 0.0409052 testing loss :0.495757
training acc: 0.997758 testing acc :0.834452 /training Loss: 0.0461103 testing loss :0.501405
training acc: 0.993834 testing acc :0.850112 /training Loss: 0.037789 testing loss :0.463464
training acc: 0.994395 testing acc :0.854586 /training Loss: 0.0446981 testing loss :0.469493
training acc: 0.992153 testing acc :0.845638 /training Loss: 0.0470385 testing loss :0.498369
training acc: 0.990471 testing acc :0.8434 /training Loss: 0.0446172 testing loss :0.532713
training acc: 0.992153 testing acc :0.832215 /training Loss: 0.0410633 testing loss :0.485993
training acc: 0.992713 testing acc :0.847875 /training Loss: 0.0444524 testing loss :0.492963
training acc: 0.988789 testing acc :0.850112 /training Loss: 0.0509959 testing loss :0.491449
training acc: 0.992713 testing acc :0.852349 /training Loss: 0.0485317 testing loss :0.487769
training acc: 0.994395 testing acc :0.847875 /training Loss: 0.04

KeyboardInterrupt: 

# sec[1-1] train raw images with naive cnn +stn ~82%

In [4]:
tf.reset_default_graph()
from layer_wrapper_stn import ceate_stn

In [6]:
# concate a naive cnn
[x,h_trans,regu_stn,h_fc_loc2,keep_prob]=ceate_stn()
[_,y_tf,y_tf_2,y_logits,regu,_] = naive_cnn(x_tensor=x,keep_prob=keep_prob)
[train_step,merged,accuracy,cross_entropy,regular_lambda,learning_rate] = evaluation(y_logits,y_tf_2,keep_prob,regu+regu_stn)

In [11]:
# create session and initialize all variables, and create writer
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#train_writer = tf.summary.FileWriter('/tmp/naive_cnn_face/train',sess.graph)
#test_writer = tf.summary.FileWriter('/tmp/naive_cnn_face/test')
count=0

In [14]:
# %% We'll now train in minibatches and report accuracy, loss:
iter_per_epoch = 15
n_epochs = 1000
train_size = len(data['X_train'])   
regular_lambda_=5e-1
indices = np.linspace(0, train_size-1, iter_per_epoch)
indices = indices.astype('int')

for epoch_i in range(n_epochs):
    for iter_i in range(iter_per_epoch - 1):
        batch_xs = data['X_train'][indices[iter_i]:indices[iter_i+1]]
        batch_ys = data['y_train'][indices[iter_i]:indices[iter_i+1]]
        sess.run(train_step, feed_dict={
            x: batch_xs, y_tf: batch_ys, learning_rate:8e-6, keep_prob:0.3, regular_lambda:regular_lambda_})
    
        if iter_i % 99 ==0and epoch_i%20 == 0:
            summary,loss,accuracy_ = sess.run([merged,cross_entropy,accuracy],
                            feed_dict={
                                x: data['X_train'],
                                y_tf: data['y_train'],
                                regular_lambda: regular_lambda_,
                                keep_prob:1
                            })

            #train_writer.add_summary(summary, count)
 
            summary,loss_val,accuracy_val = sess.run([merged,cross_entropy,accuracy],
                            feed_dict={
                                x: data['X_val'],
                                y_tf: data['y_val'],
                                regular_lambda: regular_lambda_,
                                keep_prob:1
                            })
            #test_writer.add_summary(summary, count)

            print('training acc: ' + str(accuracy_) +' testing acc :' +str(accuracy_val)+ 
                  ' /training Loss: ' + str(loss) +' testing loss :' +str(loss_val))
            count+=1
        

training acc: 0.981502 testing acc :0.814318 /training Loss: 0.142061 testing loss :0.525047
training acc: 0.985426 testing acc :0.812081 /training Loss: 0.12315 testing loss :0.520214
training acc: 0.986547 testing acc :0.823266 /training Loss: 0.122815 testing loss :0.515684
training acc: 0.984866 testing acc :0.823266 /training Loss: 0.124201 testing loss :0.520852
training acc: 0.987668 testing acc :0.823266 /training Loss: 0.121972 testing loss :0.516779
training acc: 0.987668 testing acc :0.825503 /training Loss: 0.123754 testing loss :0.513525
training acc: 0.984866 testing acc :0.814318 /training Loss: 0.122888 testing loss :0.516964
training acc: 0.985987 testing acc :0.823266 /training Loss: 0.122701 testing loss :0.519474
training acc: 0.987668 testing acc :0.818792 /training Loss: 0.121102 testing loss :0.520903
training acc: 0.985987 testing acc :0.827741 /training Loss: 0.12127 testing loss :0.513526
training acc: 0.985987 testing acc :0.821029 /training Loss: 0.121588 te

KeyboardInterrupt: 

# [sec2-1] rotation data on stn + naive cnn

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
import cv2
import random
def getrotation_img(image,angel):
    rows,cols,c = image.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angel,1)
    dst = cv2.warpAffine(image,M,(cols,rows))
    return dst

def rotation_all(X_train,angle):
    for i in xrange(X_train.shape[0]):
        angle+=random.randint(-22 ,22)
        if i==0:
            X_train_temp=np.reshape(getrotation_img(X_train[i],angle),[-1,32,32,3])
        else:
            temp=np.reshape(getrotation_img(X_train[i],angle),[-1,32,32,3])
            X_train_temp=np.concatenate((X_train_temp, temp), axis=0)
    print "the rotation output : "+str(X_train_temp.shape)
    return X_train_temp

In [3]:
# split dataset
frames = np.load("../../../whimh2.0/outfile_x.npy")
y_train = np.load("../../../whimh2.0/outfile_y.npy").astype(int)
# data is a dictionary with key X_train,y_train ,X_val,y_val
# NM labels too much
arrMH=np.where(y_train==0)[0]
arrNM=np.where(y_train==1)[0]
arrNH=np.where(y_train==2)[0]
arrME=np.where(y_train==3)[0]
arrSO=np.where(y_train==4)[0]
max_=len(arrNM)
indx=np.random.randint(max_, size=1000)
newarr=np.concatenate((arrMH,arrNM[indx],arrNH,arrME,arrSO))
frames=frames[newarr]
y_train=y_train[newarr]
print frames.shape
print y_train.shape

(2231, 32, 32, 3)
(2231,)


In [4]:
# training set
X_train=frames
X_train_0=rotation_all(frames,0)
X_train_0_1=rotation_all(frames,0)
X_train_45=rotation_all(frames,45)
X_train_45_1=rotation_all(frames,45)
X_train_90=rotation_all(frames,90)
X_train_90_1=rotation_all(frames,90)
X_train_135=rotation_all(frames,135)
X_train_135_1=rotation_all(frames,135)
X_train_180=rotation_all(frames,180)
X_train_180_1=rotation_all(frames,180)
X_train_180_2=rotation_all(frames,180)
X_train_225=rotation_all(frames,225)
X_train_225_1=rotation_all(frames,225)
X_train_270_1=rotation_all(frames,270)
X_train_270=rotation_all(frames,270)
X_train_315_1=rotation_all(frames,315)
X_train_315=rotation_all(frames,315)

the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)
the rotation output : (2231, 32, 32, 3)


In [5]:
frames=np.concatenate((X_train,X_train_0,X_train_0_1,X_train_45,X_train_45_1,X_train_90,X_train_90_1,\
                       X_train_135,X_train_180,X_train_225,X_train_135_1,X_train_180_1,X_train_180_2
                       ,X_train_225_1,X_train_270,X_train_315,X_train_270_1,X_train_315_1),axis=0)
print frames.shape

y_train=np.concatenate((y_train,y_train,y_train,y_train,y_train,y_train,\
                        y_train,y_train,y_train,y_train,y_train,y_train,\
                       y_train,y_train,y_train,y_train,y_train,y_train),axis=0)
print y_train.shape

(40158, 32, 32, 3)
(40158,)


In [2]:
frames=np.load("rotated_x.npy")
y_train=np.load("rotated_y.npy").astype(int)


In [3]:
from layer_wrapper_stn import naive_cnn,evaluation,split_data,weight_variable,bias_variable,create_stn

In [4]:
data = split_data(y_train,frames)

there are 32126 images in training set
there are 8032 images in testing set


In [5]:
tf.reset_default_graph()


In [6]:
# concate a naive cnn
[x,h_trans,regu_stn,h_fc_loc2,keep_prob] = create_stn()
[_,y_tf,y_tf_2,y_logits,regu,_] = naive_cnn(x_tensor=x,keep_prob=keep_prob)
[train_step,merged,accuracy,cross_entropy,regular_lambda,learning_rate] = \
    evaluation(y_logits,y_tf_2,keep_prob,regu+regu_stn,decay_step=1000)

In [7]:
sess = tf.Session()

In [8]:
# create session and initialize all variables, and create writer
sess.run(tf.global_variables_initializer())
#train_writer = tf.summary.FileWriter('/tmp/naive_cnn_face/train',sess.graph)
#test_writer = tf.summary.FileWriter('/tmp/stn_cnn_face/test')


In [13]:
# create pools
pools=[]
#regu
for i in [1e-1,1e-5,1e-7]:
    #learing rate
    for j in [1e-3,1e-5,1e-7]:
        #decay step
        for k in [325,750,1500,3000]:
            pools.append((i,j,k))

In [ ]:
records=dict()

for regular_lambda_,learning_rate_,decay_step_ in pools:
    tf.reset_default_graph()
    # concate a naive cnn
    [x,h_trans,regu_stn,h_fc_loc2,keep_prob] = create_stn()
    [_,y_tf,y_tf_2,y_logits,regu,_] = naive_cnn(x_tensor=x,keep_prob=keep_prob)
    [train_step,merged,accuracy,cross_entropy,regular_lambda,learning_rate] = \
        evaluation(y_logits,y_tf_2,keep_prob,regu+regu_stn,decay_step=decay_step_)
    sess = tf.Session()
    count=0
    
    # %% We'll now train in minibatches and report accuracy, loss:
    sess.run(tf.global_variables_initializer())
    print "============= current records is : " +str((regular_lambda_,learning_rate_,decay_step_))+" ================="
    
    records[(regular_lambda_,learning_rate_,decay_step_)]=10
    test_writer = tf.summary.FileWriter('/tmp/stn_cnn_face/test/'+str((regular_lambda_,learning_rate_,decay_step_)))

    iter_per_epoch = 800
    n_epochs = 300
    train_size = 32126  
    indices = np.linspace(0, train_size-1, iter_per_epoch)
    indices = indices.astype('int')
    for epoch_i in range(n_epochs):
        for iter_i in range(iter_per_epoch - 1):
            batch_xs = data['X_train'][indices[iter_i]:indices[iter_i+1]]
            batch_ys = data['y_train'][indices[iter_i]:indices[iter_i+1]]

            if iter_i % 500 == 0:
                loss,accuracy_ = sess.run([cross_entropy,accuracy],
                                feed_dict={
                                    x: batch_xs,
                                    y_tf: batch_ys,
                                    regular_lambda: regular_lambda_,
                                    keep_prob:1
                                })

                vali_xs = data['X_val']
                vali_ys = data['y_val']
                summary,loss_val,accuracy_val = sess.run([merged,cross_entropy,accuracy],
                                feed_dict={
                                    x: vali_xs,
                                    y_tf: vali_ys,
                                    regular_lambda: regular_lambda_,
                                    keep_prob:1
                                })
                test_writer.add_summary(summary, count)
                
                if(loss_val<records[(regular_lambda_,learning_rate_,decay_step_)]):
                    records[(regular_lambda_,learning_rate_,decay_step_)]=loss_val
                print('training acc: ' + str(accuracy_) +' testing acc :' +str(accuracy_val)+ 
                      ' /training Loss: ' + str(loss) +' testing loss :' +str(loss_val))
                count+=1

            sess.run(train_step, feed_dict={
                x: batch_xs, y_tf: batch_ys, learning_rate:learning_rate_, keep_prob:0.6, regular_lambda:regular_lambda_})
    sess.close()



============= current records is : (0.1, 0.001, 325) =================
training acc: 0.15 testing acc :0.120393 /training Loss: 1.60943 testing loss :1.60943
training acc: 0.55 testing acc :0.534736 /training Loss: 1.15505 testing loss :1.26873
training acc: 0.675 testing acc :0.550548 /training Loss: 1.11316 testing loss :1.21788
training acc: 0.575 testing acc :0.54009 /training Loss: 1.08327 testing loss :1.22933
training acc: 0.65 testing acc :0.554781 /training Loss: 1.08092 testing loss :1.1958
training acc: 0.55 testing acc :0.542331 /training Loss: 1.1303 testing loss :1.22691
training acc: 0.625 testing acc :0.567854 /training Loss: 1.02265 testing loss :1.15675
training acc: 0.575 testing acc :0.548182 /training Loss: 1.07618 testing loss :1.22845
training acc: 0.7 testing acc :0.57993 /training Loss: 1.07521 testing loss :1.17005
training acc: 0.55 testing acc :0.550797 /training Loss: 1.11996 testing loss :1.18893
training acc: 0.625 testing acc :0.562127 /training Loss: 1.

In [12]:
records

{(1e-05, 1e-05): 0.2827673,
 (1e-05, 0.0001): 0.2912209,
 (1e-05, 0.001): 1.4536271,
 (0.0001, 1e-05): 0.27699277,
 (0.0001, 0.0001): 0.28041306,
 (0.0001, 0.001): 0.59814215,
 (0.001, 1e-05): 0.28849912,
 (0.001, 0.0001): 0.27599043,
 (0.001, 0.001): 0.51541984}

In [ ]:
# %% We'll now train in minibatches and report accuracy, loss:
iter_per_epoch = 800
n_epochs = 200
train_size = len(data['X_train'])   
regular_lambda_=1e-3
indices = np.linspace(0, train_size-1, iter_per_epoch)
indices = indices.astype('int')

for epoch_i in range(n_epochs):
    for iter_i in range(iter_per_epoch - 1):
        batch_xs = data['X_train'][indices[iter_i]:indices[iter_i+1]]
        batch_ys = data['y_train'][indices[iter_i]:indices[iter_i+1]]
        sess.run(train_step, feed_dict={
            x: batch_xs, y_tf: batch_ys, learning_rate:3e-5, keep_prob:0.4, regular_lambda:regular_lambda_})
    
        if iter_i % 250==0:
            loss,accuracy_ = sess.run([cross_entropy,accuracy],
                            feed_dict={
                                x: data['X_train'],
                                y_tf: data['y_train'],
                                keep_prob:1
                            })

            #train_writer.add_summary(summary, count)
            #summary,loss_val,accuracy_val = sess.run([merged,cross_entropy,accuracy],
            loss_val,accuracy_val = sess.run([cross_entropy,accuracy],
                            feed_dict={
                                x: data['X_val'],
                                y_tf: data['y_val'],
                                keep_prob:1
                            })
            #test_writer.add_summary(summary, count)

            print('training acc: ' + str(accuracy_) +' testing acc :' +str(accuracy_val)+ 
                  ' /training Loss: ' + str(loss) +' testing loss :' +str(loss_val))
            count+=1
        

training acc: 0.448422 testing acc :0.44746 /training Loss: 1.60927 testing loss :1.60927
training acc: 0.448422 testing acc :0.44746 /training Loss: 1.45631 testing loss :1.45781
training acc: 0.51105 testing acc :0.517928 /training Loss: 1.35352 testing loss :1.35342
training acc: 0.514412 testing acc :0.518924 /training Loss: 1.27606 testing loss :1.27445
training acc: 0.511362 testing acc :0.517804 /training Loss: 1.26125 testing loss :1.26478
training acc: 0.543236 testing acc :0.547435 /training Loss: 1.17982 testing loss :1.18141
training acc: 0.557959 testing acc :0.561629 /training Loss: 1.13299 testing loss :1.13421
training acc: 0.562162 testing acc :0.563621 /training Loss: 1.10912 testing loss :1.11665
training acc: 0.560512 testing acc :0.559885 /training Loss: 1.11273 testing loss :1.1217
training acc: 0.57041 testing acc :0.569597 /training Loss: 1.07028 testing loss :1.07626
training acc: 0.573492 testing acc :0.576818 /training Loss: 1.0526 testing loss :1.05925
train

# [sec3-1] rotation data on stn + naive cnn, testing on un-rotation images
fail , ~0.85 